## setup

Note: Before running this notebook, populate the database:

```
$ python3 repopulate_db.py
```

All API endpoints that modify the database (rather than just fetching data from it) take a `test` argument, and if `test` is `False`, no changes to the database are made. All calls here will use the test parameter.

In [2]:
import io
import os
from pprint import pprint

import requests

In [3]:
BASE_URL = 'http://localhost:5000/'

In [4]:
def call_endpoint(method, endpoint,
                  auth=None, verbose=True, return_json=True, **kwargs):
    """
    Args:
        method ({'GET', 'PUT', 'POST', 'DELETE'})
        endpoint (str)
        auth (Tuple[str, str])
        verbose (bool)
        return_json (bool)
        **kwargs
    
    Returns:
        ``requests.response`` or JSON
    """
    url = BASE_URL + endpoint
    response = requests.request(method, url, auth=auth, **kwargs)

    if verbose is True:
        print(method, '=>', response.url)
    if return_json is True:
        return response.json()
    else:
        return response


def get_auth_token(email, password):
    return (call_endpoint('GET', 'authtoken', auth=(email, password))['token'], '')

In [11]:
auth = get_auth_token('burtondewilde@gmail.com', 'password')

GET => http://localhost:5000/authtoken


In [19]:
# call_endpoint('GET', 'studies', auth=auth,
#               params={'review_id': 1,
#                       'fields': 'citation_status,fulltext_status,citation.title,citation.authors,citation.keywords,fulltext.original_filename',
#                       'data_extraction_status': 'not_started',
#                       'order_by': 'recency', 'per_page': 10})

---

## USERS

### register a new user

input: user name, email and password

output: user info with `id` and `created_at` fields filled in by the database

In [60]:
user_info = {'name': 'Burton DeWilde',
             'email': 'burtdewilde@gmail.com',
             'password': 'password'}

call_endpoint('POST', 'register',
              json=user_info, params={'test': True})

POST => http://localhost:5000/register?test=True


{'created_at': None,
 'email': 'burtdewilde@gmail.com',
 'id': None,
 'is_admin': None,
 'is_confirmed': None,
 'last_updated': None,
 'name': 'Burton DeWilde'}

### confirm a newly-registered user

input: token generated in the `/register` endpoint and sent via email

output: nothing

This isn't an endpoint devs actually call, but a `GET` request is made to it whenever a user clicks on the link in that confirmation email.

### password reset

input: user email

output: an email with a link to reset password

In [61]:
call_endpoint('POST', 'reset',
              json={'email': 'burton@chartbeat.com', 'test': True})

POST => http://localhost:5000/reset


### authenticate an existing user

input: user email and password

output: authentication token, expires in 30 minutes

In [69]:
login_creds = ('burtondewilde@gmail.com', 'password')

response = call_endpoint('GET', 'authtoken', auth=login_creds)
auth = (response['token'], '')
auth

GET => http://localhost:5000/authtoken


('eyJleHAiOjE0Nzc4NTA3NTAsImlhdCI6MTQ3Nzg0ODk1MCwiYWxnIjoiSFMyNTYifQ.eyJpZCI6Mn0.rEG6V1G1mMqa4YpMNJsX3jw3hGelFzAqnO4nQZ95UZw',
 '')

### get user's info by id

input: user id

output: user info, or a subset specified by the `fields` parameter (**note:** `id` is always returned)

In [9]:
call_endpoint('GET', 'users/{id}'.format(id=2), auth=auth)

GET => http://localhost:5000/users/2


{'created_at': '2016-10-28T01:14:34.753711+00:00',
 'email': 'burtondewilde@gmail.com',
 'id': 2,
 'is_admin': False,
 'is_confirmed': True,
 'last_updated': '2016-10-28T01:14:34.753711+00:00',
 'name': 'Burton DeWilde'}

In [8]:
call_endpoint('GET', 'users/{id}'.format(id=2), auth=auth,
              params={'fields': 'email,name'})

GET => http://localhost:5000/users/2?fields=email%2Cname


{'email': 'burtondewilde@gmail.com', 'id': 2, 'name': 'Burton DeWilde'}

### get user's info by email

if you don't know the user's id

intput: user email

output: user info, or a subset

In [7]:
call_endpoint('GET', 'users', auth=auth,
              params={'email': 'burtondewilde@gmail.com'})

GET => http://localhost:5000/users?email=burtondewilde%40gmail.com


{'created_at': '2016-10-29T20:07:15.269526+00:00',
 'email': 'burtondewilde@gmail.com',
 'id': 2,
 'is_admin': False,
 'is_confirmed': True,
 'last_updated': '2016-10-29T20:07:15.269526+00:00',
 'name': 'Burton DeWilde'}

### get users' info by review id

this is discouraged. better to use the `/reviews/{id}/team` endpoint

In [24]:
call_endpoint('GET', 'users', auth=auth,
              params={'review_id': 1})

GET => http://localhost:5000/users?review_id=1


[{'created_at': '2016-10-29T20:07:15.269526+00:00',
  'email': 'burtondewilde@gmail.com',
  'id': 2,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-29T20:07:15.269526+00:00',
  'name': 'Burton DeWilde'},
 {'created_at': '2016-10-29T20:07:15.558762+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 3,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-29T20:07:15.558762+00:00',
  'name': 'Ray Shah'},
 {'created_at': '2016-10-29T20:07:15.840290+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 4,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-29T20:07:15.840290+00:00',
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-10-29T20:07:16.120686+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 5,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-29T20:07:16.120686+00:00',
  'name': 'Bob Minnich'},
 {'created_at': '2016-10-29T20:07:16.413553+00:00',
  'email': 'samanzaroot@gmail.com',
  'i

### delete a user

input: user id

output: nothing

**note:** this also deletes all reviews owned by user, so be very careful

In [9]:
call_endpoint('DELETE', 'users/{id}'.format(id=2), auth=auth,
              params={'test': True})

DELETE => http://localhost:5000/users/2?test=True


---

## REVIEWS

### create a new review

input: review name and description

output: review info, plus `id`, `created_at`, `status`, `num_citation_screening_reviewers`, and `num_fulltext_screening_reviewers` fields filled in by the database

In [21]:
review_info = {'name': "Burton's Great Review",
               'description': 'The greatest systematic review ever created.'}

call_endpoint('POST', 'reviews',
              json=review_info, params={'test': True},
               auth=auth)

POST => http://localhost:5000/reviews?test=True


{'created_at': None,
 'description': 'The greatest systematic review ever created.',
 'id': None,
 'last_updated': None,
 'name': "Burton's Great Review",
 'num_citation_screening_reviewers': None,
 'num_fulltext_screening_reviewers': None,
 'owner_user_id': 2,
 'status': None}

### get review's info by id

input: review id

output: review info, or a subset

In [11]:
call_endpoint('GET', 'reviews/{id}'.format(id=1), auth=auth)

GET => http://localhost:5000/reviews/1


{'created_at': '2016-10-29T20:07:16.706856+00:00',
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\n\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conser

### get info for all reviews visible to authenticated user

input: nothing

output: list of reviews' info, or subsets thereof

In [12]:
call_endpoint('GET', 'reviews', auth=auth,
              params={'fields': 'name,status'})

GET => http://localhost:5000/reviews?fields=name%2Cstatus


[{'id': 1, 'name': 'Conservation International Demo', 'status': 'active'}]

### change a review's info / settings

input: review id and field: value pairs to update

output: updated review info

In [13]:
modified_review_info = {'name': 'CI Demo',
                        'num_citation_screening_reviewers': 1}

call_endpoint('PUT', 'reviews/{id}'.format(id=1), auth=auth,
              json=modified_review_info, params={'test': True})

PUT => http://localhost:5000/reviews/1?test=True


{'created_at': '2016-10-29T20:07:16.706856+00:00',
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\n\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conser

### delete a review

input: review id

output: nothing

In [23]:
call_endpoint('DELETE', 'reviews/{id}'.format(id=1), auth=auth,
              params={'test': True}, return_json=False)

DELETE => http://localhost:5000/reviews/1?test=True


<Response [200]>

---

## REVIEW TEAMS

### get the team of users on a review

input: review id

output: list of users on the review; owner is specially noted by `is_owner` field

In [18]:
call_endpoint('GET', 'reviews/{id}/team'.format(id=1), auth=auth)

GET => http://localhost:5000/reviews/1/team


[{'created_at': '2016-10-28T01:14:34.753711+00:00',
  'email': 'burtondewilde@gmail.com',
  'id': 2,
  'is_admin': False,
  'is_confirmed': True,
  'is_owner': True,
  'last_updated': '2016-10-28T01:14:34.753711+00:00',
  'name': 'Burton DeWilde'},
 {'created_at': '2016-10-28T01:14:35.067432+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 3,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.067432+00:00',
  'name': 'Ray Shah'},
 {'created_at': '2016-10-28T01:14:35.365292+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 4,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.365292+00:00',
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-10-28T01:14:35.662913+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 5,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.662913+00:00',
  'name': 'Bob Minnich'},
 {'created_at': '2016-10-28T01:14:35.957847+00:00',
  'email': 'samanzar

### add or remove a user as a collaborator on a review

input: review id, user id, action type

output: updated list of users on the review

In [22]:
call_endpoint('PUT', 'reviews/{id}/team'.format(id=1),
              json={'user_id': 3, 'action': 'add'},
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/team?test=True


{'error': 'forbidden', 'message': '<User(id=3)> is already on this review'}

In [23]:
call_endpoint('PUT', 'reviews/{id}/team'.format(id=1),
              json={'user_id': 3, 'action': 'remove'},
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/team?test=True


[{'created_at': '2016-10-28T01:14:34.753711+00:00',
  'email': 'burtondewilde@gmail.com',
  'id': 2,
  'is_admin': False,
  'is_confirmed': True,
  'is_owner': True,
  'last_updated': '2016-10-28T01:14:34.753711+00:00',
  'name': 'Burton DeWilde'},
 {'created_at': '2016-10-28T01:14:35.067432+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 3,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.067432+00:00',
  'name': 'Ray Shah'},
 {'created_at': '2016-10-28T01:14:35.365292+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 4,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.365292+00:00',
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-10-28T01:14:35.662913+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 5,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.662913+00:00',
  'name': 'Bob Minnich'},
 {'created_at': '2016-10-28T01:14:35.957847+00:00',
  'email': 'samanzar

### assign user as new review owner

input: review id, user id, action = 'make_owner'

output: updated list of users on the review

In [25]:
call_endpoint('PUT', 'reviews/{id}/team'.format(id=1),
              json={'user_id': 3, 'action': 'make_owner'},
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/team?test=True


[{'created_at': '2016-10-28T01:14:34.753711+00:00',
  'email': 'burtondewilde@gmail.com',
  'id': 2,
  'is_admin': False,
  'is_confirmed': True,
  'is_owner': True,
  'last_updated': '2016-10-28T01:14:34.753711+00:00',
  'name': 'Burton DeWilde'},
 {'created_at': '2016-10-28T01:14:35.067432+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 3,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.067432+00:00',
  'name': 'Ray Shah'},
 {'created_at': '2016-10-28T01:14:35.365292+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 4,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.365292+00:00',
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-10-28T01:14:35.662913+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 5,
  'is_admin': False,
  'is_confirmed': True,
  'last_updated': '2016-10-28T01:14:35.662913+00:00',
  'name': 'Bob Minnich'},
 {'created_at': '2016-10-28T01:14:35.957847+00:00',
  'email': 'samanzar

---

## REVIEW PLANS

### get a review plan

input: review id, optional fields to return

output: review plan

In [26]:
call_endpoint('GET', 'reviews/{id}/plan'.format(id=1), auth=auth)

GET => http://localhost:5000/reviews/1/plan


{'boolean_search_query': '(("wellbeing" OR "well-being" OR "well being") OR ("ecosystem service" OR "ecosystem services") OR "nutrition" OR ("skill" OR "skills") OR ("empower" OR "empowering") OR ("clean water" OR "food security") OR ("livelihoods" OR "livelihood") OR ("resilience" OR "vulnerability") OR ("capital" OR "social capital") OR ("attitude" OR "attitudes") OR ("perception" OR "perceptions") OR ("health" OR "human health") OR ("human capital" OR "knowledge") OR "traditional knowledge")\nAND\n(("marine" OR "freshwater") OR "coastal" OR ("forest" OR "forests" OR "forestry") OR ("ecosystem" OR "ecosystems") OR "species" OR ("habitat" OR "habitats") OR "biodiversity" OR ("sustainable" OR "sustainability") OR ("ecology" OR "ecological") OR "integrated" OR "landscape" OR "seascape" OR ("coral reef" OR "coral reefs") OR ("natural resources" OR "natural resource"))\nAND\n(("human" OR "humans" OR "humanity") OR "people" OR ("person" OR "persons") OR ("community" OR "communities") OR ("

### add or modify parts of a review plan

input: review id, field: value pairs to modify

output: modified review plan

In [27]:
update = {
    'fields': 'objective,pico',
    'objective': 'foo bar bat baz',
    'pico': {'population': 'lots of people',
             'intervention': 'we did a thing',
             'comparison': 'we did not do a thing',
             'outcome': 'things got better'}
    }

call_endpoint('PUT', 'reviews/{id}/plan'.format(id=1),
              json=update,
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/plan?test=True


{'boolean_search_query': '(("wellbeing" OR "well-being" OR "well being") OR ("ecosystem service" OR "ecosystem services") OR "nutrition" OR ("skill" OR "skills") OR ("empower" OR "empowering") OR ("clean water" OR "food security") OR ("livelihoods" OR "livelihood") OR ("resilience" OR "vulnerability") OR ("capital" OR "social capital") OR ("attitude" OR "attitudes") OR ("perception" OR "perceptions") OR ("health" OR "human health") OR ("human capital" OR "knowledge") OR "traditional knowledge")\nAND\n(("marine" OR "freshwater") OR "coastal" OR ("forest" OR "forests" OR "forestry") OR ("ecosystem" OR "ecosystems") OR "species" OR ("habitat" OR "habitats") OR "biodiversity" OR ("sustainable" OR "sustainability") OR ("ecology" OR "ecological") OR "integrated" OR "landscape" OR "seascape" OR ("coral reef" OR "coral reefs") OR ("natural resources" OR "natural resource"))\nAND\n(("human" OR "humans" OR "humanity") OR "people" OR ("person" OR "persons") OR ("community" OR "communities") OR ("

### delete an entire review plan

although why you would ever do this, I don't know... in fact we probably don't want to enable a "clear" option on this screen. nevertheless!

input: review id

output: nothing

In [29]:
call_endpoint('DELETE', 'reviews/{id}/plan'.format(id=1), auth=auth,
              params={'test': True}, return_json=False)

DELETE => http://localhost:5000/reviews/1/plan?test=True


<Response [204]>

---

## CITATIONS

### create a single citation

user manually inputs citation information, one at a time; useful for "gray literature", where a proper RIS or BibTex file isn't readily available

input: citation's information, bare minimum is review id but that's a low bar to pass

output: saved citation info, with `id`, `created_at`, and `status` filled in by the database

In [71]:
citation_to_add = {
    'review_id': 1,
    'title': 'My Great Citation',
    'authors': 'B. DeWilde',
    'source_type': 'database',
    'source_name': 'scopus',
    }

call_endpoint('POST', 'citations', auth=auth,
              json=citation_to_add,
              params={'test': True})

POST => http://localhost:5000/citations?test=True


''

### import many citations in a file

input: review id and full path on disk of citations file to upload

output: nothing

In [72]:
filepaths = ['../colandr_data/citations/ci-full-collection-group1.ris',
             '../colandr_data/citations/ci-full-collection-group2.ris']
# filepaths = ['../data/raw/citation_files/phase_2_demo_citations.ris']

for filepath in filepaths:
    filename = os.path.split(filepath)[-1]
    result = call_endpoint(
        'POST', 'citations/imports', auth=auth,
        data={'review_id': 1, 'status': 'included',
              'source_type': 'database', 'source_name': 'scopus', 'source_url': None,
              'test': True},
        files={'uploaded_file': (filename, io.open(filepath, mode='rb'))})
    print(result)

POST => http://localhost:5000/citations/imports
None


### get a single citation

input: citation id, fields to return (optional)

output: citation info

In [72]:
citation_id = 10

call_endpoint('GET', 'citations/{id}'.format(id=citation_id),
              params={'fields': None},
              auth=auth)

GET => http://localhost:5000/citations/10


{'abstract': 'This chapter highlights contemporary challenges in Atlantic salmon research, and discusses the relationship between research and conservation. The myriad ecological and genetic studies of salmon that have been undertaken around the Northern Hemisphere over the past century and more have powerfully advanced our understanding of salmon biology. However, knowledge alone will not save salmon; only actions will. Indeed, we know enough now about salmon ecology and genetics to reverse or at least reduce the decline of many populations or protect those that have not declined. Nonetheless, we still lack important knowledge related to several aspects of salmon ecology and life - history strategies, particularly in the sea. In addition, we do not know to what degree increasing smolt production in fresh water can compensate for problems salmon face in the marine phase of their lives. Better cooperation, improved methods and technology and more carefully designed studies will help to 

In [73]:
citation_id = 10

call_endpoint('GET', 'citations/{id}'.format(id=citation_id),
              params={'fields': 'title,abstract,keywords,authors'},
              auth=auth)

GET => http://localhost:5000/citations/10?fields=title%2Cabstract%2Ckeywords%2Cauthors


{'abstract': 'This chapter highlights contemporary challenges in Atlantic salmon research, and discusses the relationship between research and conservation. The myriad ecological and genetic studies of salmon that have been undertaken around the Northern Hemisphere over the past century and more have powerfully advanced our understanding of salmon biology. However, knowledge alone will not save salmon; only actions will. Indeed, we know enough now about salmon ecology and genetics to reverse or at least reduce the decline of many populations or protect those that have not declined. Nonetheless, we still lack important knowledge related to several aspects of salmon ecology and life - history strategies, particularly in the sea. In addition, we do not know to what degree increasing smolt production in fresh water can compensate for problems salmon face in the marine phase of their lives. Better cooperation, improved methods and technology and more carefully designed studies will help to 

### get many citations

input: review id (required), comma-delimited list of fields to return, several filtering options, ordering options, and pagination parameters

output: list of N = `per_page` citations

In [116]:
# sort by recency
call_endpoint('GET', 'citations',
              params={'review_id': 1,
                      'fields': 'status,title',
                      'status': 'pending',
                      'tag': None,
                      'tsquery': 'fisheries',
                      'order_by': 'recency',
                      'order_dir': 'ASC', 'per_page': 10},
              auth=auth)

GET => http://localhost:5000/citations?per_page=10&review_id=1&fields=status%2Ctitle&status=pending&order_by=recency&order_dir=ASC&tsquery=fisheries


[{'id': 33,
  'status': 'screened_once',
  'title': 'Molecular provenance analysis for shy and white-capped albatrosses killed by fisheries interactions in Australia, New Zealand, and South Africa'},
 {'id': 81,
  'status': 'excluded',
  'title': 'Economics of date palm agriculture in the sultanate of Oman, current situation and future prospects'},
 {'id': 91,
  'status': 'not_screened',
  'title': 'Implications of community and stakeholder perceptions of the marine environment and its conservation for MPA management in a small Azorean island'},
 {'id': 315,
  'status': 'not_screened',
  'title': "An environmentalist's perspective on responsible fisheries: the need for holistic approaches"},
 {'id': 320,
  'status': 'screened_once',
  'title': 'Community fishery resources management on Malalison Island, Philippines: R & D framework, interventions, and policy implications'},
 {'id': 390,
  'status': 'not_screened',
  'title': 'Indiscriminate exploitation of wild prawn postlarvae in the 

In [117]:
# sort by recency
call_endpoint('GET', 'citations',
              params={'review_id': 1,
                      'fields': 'status,title',
                      'status': 'pending',
                      'tag': 'some tag',
                      'tsquery': 'fisheries',
                      'order_by': 'recency',
                      'order_dir': 'ASC', 'per_page': 10},
              auth=auth)

GET => http://localhost:5000/citations?per_page=10&review_id=1&fields=status%2Ctitle&status=pending&order_by=recency&order_dir=ASC&tag=some+tag&tsquery=fisheries


[]

In [118]:
# sort by relevance
call_endpoint('GET', 'citations',
              params={'review_id': 1,
                      'fields': 'title,status',
                      'tag': None,
                      'order_by': 'relevance',
                      'order_dir': 'DESC', 'per_page': 10},
              auth=auth)

GET => http://localhost:5000/citations?review_id=1&fields=title%2Cstatus&order_by=relevance&order_dir=DESC&per_page=10


[{'id': 11074,
  'status': 'included',
  'title': 'Certification of a Community-based Forest Enterprise for Improving Institutional Management and Household Income: A Case from Southeast Sulawesi, Indonesia'},
 {'id': 5810,
  'status': 'excluded',
  'title': 'Trends in development of coastal area management in tropical countries: From central to community orientation'},
 {'id': 2960,
  'status': 'excluded',
  'title': 'Joint forest management in India: Experiences of two decades'},
 {'id': 18616,
  'status': 'screened_once',
  'title': "Linking livelihoods and conservation: An examination of local residents' perceived linkages between conservation and livelihood benefits around Nepal's Chitwan National Park"},
 {'id': 2270,
  'status': 'excluded',
  'title': 'Bureaucratic barriers limit local participatory governance in protected areas in Costa Rica'},
 {'id': 1997,
  'status': 'screened_once',
  'title': 'Looking back and looking ahead: Local empowerment and governance in the Annapurn

### get citation import history

input: review id

output: list of citation imports (see POSTs to `citations/upload` described above)

In [7]:
call_endpoint('GET', 'citations/upload',
              params={'review_id': 1}, auth=auth)

GET => http://localhost:5000/citations/upload?review_id=1


[{'created_at': '2016-10-16T22:36:35.692648+00:00',
  'data_source': {},
  'id': 1,
  'num_records': 13332,
  'record_type': 'citation',
  'review_id': 1,
  'status': 'not_screened',
  'user': {'created_at': '2016-10-16T22:36:33.970063+00:00',
   'email': 'burtondewilde@gmail.com',
   'id': 2,
   'name': 'Burton DeWilde'},
  'user_id': 2},
 {'created_at': '2016-10-16T22:36:49.780447+00:00',
  'data_source': {},
  'id': 2,
  'num_records': 15376,
  'record_type': 'citation',
  'review_id': 1,
  'status': 'not_screened',
  'user': {'created_at': '2016-10-16T22:36:33.970063+00:00',
   'email': 'burtondewilde@gmail.com',
   'id': 2,
   'name': 'Burton DeWilde'},
  'user_id': 2}]

---

## Screen Citations

### submit a screening

input: citation id (in URL), status, and (if excluded) list of exclude reasons

output: citation screening info, plus id, created_at, review_id, and user_id fields filled in by the database

In [29]:
citation_id = 1
screening_1 = {'status': 'included'}
screening_2 = {'status': 'excluded', 'exclude_reasons': ['foo', 'bar']}

result_1 = call_endpoint(
    'POST', 'citations/{}/screenings'.format(citation_id),
    json=screening_1, params={'test': True},
    auth=('caugustin@rsmas.miami.edu', 'password'))

pprint(result_1)
print()

result_2 = call_endpoint(
    'POST', 'citations/{}/screenings'.format(citation_id),
    json=screening_2, params={'test': True},
    auth=('rcm2164@columbia.edu', 'password'))

pprint(result_2)

POST => http://localhost:5000/citations/1/screenings?test=True
{'citation_id': 1,
 'created_at': None,
 'exclude_reasons': None,
 'id': None,
 'review_id': 1,
 'status': 'included',
 'user_id': 4}

POST => http://localhost:5000/citations/1/screenings?test=True
{'citation_id': 1,
 'created_at': None,
 'exclude_reasons': ['foo', 'bar'],
 'id': None,
 'review_id': 1,
 'status': 'excluded',
 'user_id': 5}


### delete a screening

user is able to delete his or her own screenings; we have to decide how to surface this functionality in the app

input: citation id (via URL)

output: nothing

In [30]:
citation_id = 1

call_endpoint('DELETE', 'citations/{}/screenings'.format(citation_id),
              params={'test': True},
              auth=('burtondewilde@gmail.com', 'password'))

call_endpoint('DELETE', 'citations/{}/screenings'.format(citation_id),
              params={'test': True},
              auth=('rayshah@thinkdesign.com', 'password'))

DELETE => http://localhost:5000/citations/1/screenings?test=True
DELETE => http://localhost:5000/citations/1/screenings?test=True


### get existing screenings for a single citation

input: citation id, fields of screenings to return

output: resulting screenings

In [31]:
citation_id = 2

call_endpoint('GET', 'citations/{}/screenings'.format(citation_id),
              params={'fields': None},
              auth=('burtondewilde@gmail.com', 'password'))

GET => http://localhost:5000/citations/2/screenings


[{'citation_id': 2,
  'created_at': '2016-10-10T21:52:53.967122+00:00',
  'exclude_reasons': ['location', 'outcome'],
  'id': 21653,
  'review_id': 1,
  'status': 'excluded',
  'user_id': 3},
 {'citation_id': 2,
  'created_at': '2016-10-10T21:52:45.021692+00:00',
  'exclude_reasons': ['location'],
  'id': 1,
  'review_id': 1,
  'status': 'excluded',
  'user_id': 2}]

---

## FULLTEXTS

fulltexts are created automatically when a citation is marked as "included", and removed automatically when a citation is un-marked as "included" (which is actually more of an edge case). the api endpoints are mostly analogous to those for citations.

### get a single fulltext

input: fulltext id, fields to return (optional)

output: fulltext info; **note:** this also includes associated citation and screenings, if fields are not specified

In [96]:
fulltext_id = 44

call_endpoint('GET', 'fulltexts/{id}'.format(id=fulltext_id),
              params={'fields': None},
              auth=auth)

GET => http://localhost:5000/fulltexts/44


{'created_at': '2016-10-30T18:49:27.515173+00:00',
 'filename': '44.pdf',
 'id': 44,
 'last_updated': '2016-10-30T18:49:27.515173+00:00',
 'review_id': 1,
 'screenings': [{'created_at': '2016-10-30T18:49:53.431063+00:00',
   'exclude_reasons': ['in-situ', 'intervention type'],
   'fulltext_id': 44,
   'id': 1,
   'last_updated': '2016-10-30T18:49:53.431063+00:00',
   'review_id': 1,
   'status': 'excluded',
   'user_id': 2},
  {'created_at': '2016-10-30T18:49:54.564882+00:00',
   'exclude_reasons': ['undefined pop.', 'location'],
   'fulltext_id': 44,
   'id': 541,
   'last_updated': '2016-10-30T18:49:54.564882+00:00',
   'review_id': 1,
   'status': 'excluded',
   'user_id': 3}],
 'status': 'excluded'}

In [98]:
fulltext_id = 44

call_endpoint('GET', 'fulltexts/{id}'.format(id=fulltext_id),
              params={'fields': 'status,filename,study.citation.title'},
              auth=auth)

GET => http://localhost:5000/fulltexts/44?fields=status%2Cfilename%2Cstudy.citation.title


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### get many fulltexts

input: review id (required), fields to return for each (optional), plus filtering, ordering, and pagination options

output: list of N = `per_page` fulltexts

In [92]:
# sort by recency
call_endpoint('GET', 'fulltexts',
              params={'review_id': 1,
                      'fields': 'status,citation.title',
                      'status': 'pending',
                      'tag': None,
                      'tsquery': None,
                      'order_by': 'recency',
                      'order_dir': 'ASC', 'per_page': 10},
              auth=auth)

GET => http://localhost:5000/fulltexts?per_page=10&review_id=1&fields=status%2Ccitation.title&status=pending&order_by=recency&order_dir=ASC


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### upload a PDF of TXT file for a fulltext

input: fulltext id, file to upload

output: fulltext info, with `filename` and `content` included

In [94]:
fulltext_id = 1
filepath = '../references/Bottrill_et al_2014_systematic protocol.pdf'
filename = 'Bottrill_et al_2014_systematic protocol.pdf'

call_endpoint('POST', 'fulltexts/{id}/upload'.format(id=fulltext_id),
              params={'test': True},
              files={'uploaded_file': (filename, io.open(filepath, mode='rb'))},
              auth=auth)

POST => http://localhost:5000/fulltexts/1/upload?test=True


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### get a fulltext upload

input: fulltext id

output: pdf data, as bytes (must be displayed somehow -- which I have failed to do below)

In [36]:
from IPython.display import IFrame, Image, HTML

In [37]:
fulltext_id = 1

pdf_data = call_endpoint('GET', 'fulltexts/{id}/upload'.format(id=fulltext_id),
                         auth=auth, return_json=False).content

IFrame(pdf_data, width=300, height=400)

GET => http://localhost:5000/fulltexts/1/upload


---

## FULLTEXT SCREENING

almost exactly the same as citation screening

---

## DATA EXTRACTION

In [79]:
call_endpoint('DELETE', 'fulltexts/{id}/extracted_data'.format(id=fulltext_id),
              params={'labels': 'outcome_type', 'test': False},
              auth=auth, return_json=False)

DELETE => http://localhost:5000/fulltexts/5/extracted_data?test=False&labels=outcome_type


<Response [204]>

In [80]:
call_endpoint('GET', 'fulltexts/{id}/extracted_data'.format(id=fulltext_id),
              auth=auth)

GET => http://localhost:5000/fulltexts/5/extracted_data


{'created_at': '2016-10-13T02:00:23.321169+00:00',
 'extracted_data': [{'label': 'intervention_type',
   'value': ['area management', 'area protection']}],
 'fulltext_id': 5,
 'id': 99,
 'review_id': 1}

In [77]:
fulltext_id = 5
extracted_data = [{'label': 'outcome_type', 'value': ['subjective well-being']},
                  {'label': 'intervention_type', 'value': ['area management', 'area protection']}]

call_endpoint('PUT', 'fulltexts/{id}/extracted_data'.format(id=fulltext_id),
              json=extracted_data, params={'test': False},
              auth=auth)

PUT => http://localhost:5000/fulltexts/5/extracted_data?test=False


{'created_at': '2016-10-13T02:00:23.321169+00:00',
 'extracted_data': [{'label': 'outcome_type',
   'value': ['subjective well-being']},
  {'label': 'intervention_type',
   'value': ['area management', 'area protection']}],
 'fulltext_id': 5,
 'id': 99,
 'review_id': 1}

---

## EXPORT

In [236]:
review_id = 1

call_endpoint('GET', 'reviews/{id}/export_prisma'.format(id=review_id),
              auth=auth)

INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): localhost


GET => http://localhost:5000/reviews/1/export_prisma


{'exclude_reason_counts': {'in-situ': 173,
  'intervention type': 149,
  'location': 198,
  'outcome': 158,
  'undefined pop.': 150},
 'num_excluded_citations': 15872,
 'num_excluded_fulltexts': 106,
 'num_screened_citations': 16586,
 'num_screened_fulltexts': 210,
 'num_studies_by_source': {'database': 28708},
 'num_studies_data_extracted': 0,
 'num_unique_studies': 28343}

In [14]:
review_id = 1

response = call_endpoint(
    'GET', 'reviews/{id}/export_studies'.format(id=review_id),
    auth=auth, return_json=False)
print('headers:', response.headers)

INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): localhost


GET => http://localhost:5000/reviews/1/export_studies
headers: {'Date': 'Tue, 01 Nov 2016 03:03:59 GMT', 'Server': 'Werkzeug/0.11.11 Python/3.5.2', 'Content-type': 'text/csv', 'Content-Length': '55563838'}


In [15]:
with io.open('/Users/burtondewilde/Desktop/foo.csv', mode='w') as f:
    f.write(response.text)

--- 

## testing...

In [87]:
# # sort by relevance
# call_endpoint('GET', 'citations',
#               params={'review_id': 1,
#                       'fields': 'authors,title,journal_name,volume,issue_number,pub_year,abstract,keywords,tags',
#                       'tsquery': None,
#                       'order_by': 'relevance',
#                       'order_dir': 'DESC', 'per_page': 10},
#               auth=('burtondewilde@gmail.com', 'password'))

In [35]:
# %%time

from sqlalchemy import select

from colandr import create_app, db
from colandr.models import *

app = create_app('dev')

with app.app_context():
    
#     query = db.session.query(Study).join(Citation, Citation.id == Study.id)\
#         .limit(10)\
#         .with_entities(Study.id, Citation.text_content)

    citation = db.aliased(Citation)
    query = db.session.query(Study).join(citation, Study.citation)\
        .filter(Study.review_id == 1)\
        .filter(citation.text_content.match('fisheries'))
    
#     result = query.all()
#     print(result)